In [1]:
import pandas as pd
from methods import *

%load_ext autoreload
%autoreload 2
print('done')

In [2]:
def select_columns_from_dataset():
    path = r'confusion-reports\ics_fc\round3-holdout-dataset\round3-holdout-dataset_fc_square30-random_filters.csv'
    df = pd.read_csv(path)
#     for col_to_delete in ['square30_r_mean_diff', 'square30_r_std_diff', 'square30_r_mean', 'square30_r_std']:
#     for col_to_delete in ['square30_r_mean_diff', 'square30_r_std_diff', 'square30_r_mean', 'square30_r_std']:
#         del df[col_to_delete]
    df_cols = df.columns
    for col_to_delete in df_cols:
        if 'mean' in col_to_delete or 'std' in col_to_delete:
            del df[col_to_delete]
    df.to_csv(r'confusion-reports\round3-holdout-columns.csv', index=False)
#     df.to_csv(f'{path.replace(".csv", "")}_changed.csv', index=False)

def merge_datasets():
#     for size in [10, 20, 25, 30, 35, 40, 45, 50]:
#     for size in [25, 30, 35, 40, 45, 50]:
    for size in [30]:
        path = fr'confusion-reports\ics_fc\round3-holdout-dataset\round3-holdout-dataset_fc_square-{size}-gray_filters.csv'
#         path = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-{size}-random_filters.csv'
#         path = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}_filters.csv'
#         path = fr'confusion-reports\conf_mat\round3-train-dataset_fc_square-{size}-random_filters_h_kl.csv'
        df_to_modify_and_save = pd.read_csv(path)
        df_to_copy_columns_from = pd.read_csv(fr'confusion-reports\round3-holdout-columns.csv')
        df_to_modify_and_save['architecture_code'] = df_to_copy_columns_from['architecture_code']
#         for col in [f'square{size}_mean_diff', f'square{size}_std_diff', f'square{size}_mean', f'square{size}_std']:
#             df_to_modify_and_save[col] = df_to_copy_columns_from[col]
        new_order_for_columns = [
            'model_name', 'model_architecture', 'architecture_code', 'model_label', 'trigger_type_aux',

            f'square{size}_mean_diff', f'square{size}_std_diff',
#             f'square{size}_r_mean_diff', f'square{size}_r_std_diff',
            'gotham_mean_diff', 'gotham_std_diff',
            'kelvin_mean_diff', 'kelvin_std_diff',
            'lomo_mean_diff', 'lomo_std_diff',
            'nashville_mean_diff', 'nashville_std_diff',
            'toaster_mean_diff', 'toaster_std_diff',

            'clean_mean', 'clean_std',
            f'square{size}_mean', f'square{size}_std',
#             f'square{size}_r_mean', f'square{size}_r_std',
            'gotham_mean', 'gotham_std',
            'kelvin_mean', 'kelvin_std',
            'lomo_mean', 'lomo_std',
            'nashville_mean', 'nashville_std',
            'toaster_mean', 'toaster_std',
            
#             'h_clean', 'kl_clean',
#             'h_square30', 'kl_square30',
#             'h_gotham', 'kl_gotham',
#             'h_kelvin', 'kl_kelvin',
#             'h_lomo', 'kl_lomo',
#             'h_nashville', 'kl_nashville',
#             'h_toaster', 'kl_toaster',

#             'trigger_color', 
            'num_classes',
        ]
        df_to_modify_and_save = df_to_modify_and_save[new_order_for_columns]
        df_to_modify_and_save.to_csv(path, index=False)

def correct_column_in_dataset():
    df_with_correct_column = pd.read_csv(r'confusion-reports\ics_svm\round3-train-dataset\round3-columns.csv')
    column = 'trigger_type_aux'
    for size in [10, 15, 20, 25,30, 35, 40, 45, 50]:
        file = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}-filters_all-classes_gray.csv'
        df_to_be_corrected = pd.read_csv(file)
        df_to_be_corrected[column] = df_with_correct_column[column]
        df_to_be_corrected.to_csv(file, index=False)

def create_architecture_code():
    df_columns = pd.read_csv(r'confusion-reports\round3-columns.csv')
    list_codes = []
    for i in range(len(df_columns)):
        model_architecture = df_columns['model_architecture'].iloc[i]
        arch_code = encode_architecture(model_architecture)
        if arch_code is None:
            print(f'Error at index {i}')
            break
        list_codes.append(arch_code)
    df_columns['architecture_code'] = list_codes
    df_columns = df_columns[['model_name', 'poisoned', 'model_architecture', 'architecture_code', 'trigger_type_aux', 'trigger_type', 'instagram_filter_type', 'polygon_side_count']]
    df_columns.to_csv(r'confusion-reports\round3-columns.csv', index=False)

def create_columns_csv_from_metadata():
    path_metadata = r'confusion-reports\METADATA-holdout.csv'
    df = pd.DataFrame(columns=['model_name','poisoned','model_architecture','architecture_code','trigger_type_aux','trigger_type','instagram_filter_type','polygon_side_count'])
    n_df = 0
    metadata = pd.read_csv(path_metadata)
    for i in range(len(metadata)):
        model_name = metadata['model_name'].iloc[i]
        poisoned = metadata['poisoned'].iloc[i]
        model_architecture = metadata['model_architecture'].iloc[i]
        architecture_code = encode_architecture(model_architecture)
        trigger_type = metadata['trigger_type'].iloc[i]
        polygon_side_count = metadata['polygon_side_count'].iloc[i]
        instagram_filter_type = metadata['instagram_filter_type'].iloc[i]
        trigger_type_aux = get_trigger_type_aux_value(trigger_type, polygon_side_count, instagram_filter_type)
        df.loc[n_df] = [model_name, poisoned, model_architecture, architecture_code, trigger_type_aux, trigger_type, instagram_filter_type, polygon_side_count]
        n_df += 1
    df.to_csv(r'confusion-reports\round3-holdout-columns.csv', index=False)

def add_column_for_backdoor_type():
#     path = fr'confusion-reports\ics_fc\round3-holdout-dataset\round3-holdout-dataset_fc_square-30-gray_filters.csv'
    path = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_filters.csv'
    df = pd.read_csv(path)
    column_backdoor_code = []
    for i in range(len(df)):
        trigger_type_aux = df['trigger_type_aux'].iloc[i]
        code = encode_backdoor(trigger_type_aux)
        column_backdoor_code.append(code)    
    df.insert(3, 'backdoor_code', column_backdoor_code)
    df.to_csv(path, index=False)

def add_backdoor_code_2_column():
    path = r'confusion-reports\ics_fc\round4-train-dataset\round4-train-dataset_fc_synthetic_polygon-all-gray_filters_T=0.0.csv'
    df = pd.read_csv(path)
    col_backd_code_2 = []
    for i in range(len(df)):
        backd_0 = df['backdoor_code_0'].iloc[i]
        backd_1 = df['backdoor_code_1'].iloc[i]
        backd_2 = None
        if backd_0 == 0 and backd_1 == 0:
            backd_2 = '(0,)'
        else:
            if backd_0 == 0:
                backd_2 = str((backd_1,))
            elif backd_1 == 0:
                backd_2 = str((backd_0,))
            else:
                backd_2 = str(tuple(sorted([backd_0, backd_1]))).replace(' ', '')
        col_backd_code_2.append(backd_2)
    df.insert(5, 'backdoor_code_2', col_backd_code_2)
    df.to_csv(path, index=False)

def normalize_confusion_distribution_by_num_classes():
    path = r'confusion-reports/ics_fc/round4-train-dataset/round4-train-dataset_fc_synthetic_polygon-all-gray_filters_T=0.5_normalized_by_num_classes.csv'
    df = pd.read_csv(path)
    for c in df.columns:
        if 'mean' in c:
            df[c] /= df['num_classes']
        if 'std' in c:
            df[c] /= (df['num_classes'] ** 2)
    df.to_csv(path, index=False)
        
# merge_datasets()
# select_columns_from_dataset()
# correct_column_in_dataset()
# create_architecture_code()
# create_columns_csv_from_metadata()
# add_column_for_backdoor_type()
add_backdoor_code_2_column()
# normalize_confusion_distribution_by_num_classes()
print('done')

done
